<a href="https://colab.research.google.com/github/adowdell18/Work_Samples_10.2021/blob/main/AD_Digits_Work_Sample_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Digits Work Sample - Machine Learning

Before you start, please review the attached document with tips regarding the setup, and what to do when you get stuck or when the time is up.



# DBpedia Classification Problem

For this work sample, you'll be using the [DBpedia14 dataset](https://huggingface.co/datasets/dbpedia_14). The dataset consists of title, content (both string values) and a label (int) which refers to the label group.

The classes are

* Company
* EducationalInstitution
* Artist
* Athlete
* OfficeHolder
* MeanOfTransportation
* Building
* NaturalPlace
* Village
* Animal
* Plant
* Album
* Film
* WrittenWork

Below we have set you up with the initial dataset download and conversion to a Pandas dataframes for the training and test set.

The work sample contains 4 tasks which all need to be solved in the context of the datasets.  

In [ ]:

# Installing required packages
!pip install -q --user datasets
!pip install datasets
!pip install pyLDAvis
!pip install pyLDAvis.gensim

# Restart the kernel after installation of pip packages

ERROR: Could not find a version that satisfies the requirement pyLDAvis.gensim (from versions: none)
ERROR: No matching distribution found for pyLDAvis.gensim


In [ ]:
import datasets
from datasets import load_dataset
from sklearn.datasets  import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import glob
import nltk
import spacy
import random
import gensim
import pyLDAvis.gensim_models
import itertools
spacy.load('en')
from spacy.lang.en import English
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import cufflinks
import pickle
pd.options.display.max_columns = 30
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE
from collections import Counter
from nltk.stem.wordnet import WordNetLemmatizer





In [ ]:
from datasets import load_dataset
from __future__ import print_function
from gensim import corpora

In [ ]:
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)

In [ ]:
parser = English()
spacy.load('en')
nltk.download('wordnet')
nltk.download('punkt')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df_train = load_dataset('dbpedia_14', split='train').to_pandas()
df_test = load_dataset('dbpedia_14', split='test').to_pandas()

Downloading:   0%|          | 0.00/2.14k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset d_bpedia14 downloaded and prepared to /root/.cache/huggingface/datasets/d_bpedia14/dbpedia_14/2.0.0/7f0577ea0f4397b6b89bfe5c5f2c6b1b420990a1fc5e8538c7ab4ec40e46fa3e. Subsequent calls will reuse this data.


Reusing dataset d_bpedia14 (/root/.cache/huggingface/datasets/d_bpedia14/dbpedia_14/2.0.0/7f0577ea0f4397b6b89bfe5c5f2c6b1b420990a1fc5e8538c7ab4ec40e46fa3e)


In [ ]:
df_test.head()

,label,title,content
0,0,TY KU,TY KU /taɪkuː/ is an American alcoholic bever...
1,0,Odd Lot Entertainment,OddLot Entertainment founded in 2001 by longt...
2,0,Henkel,Henkel AG & Company KGaA operates worldwide w...
3,0,GOAT Store,The GOAT Store (Games Of All Type Store) LLC ...
4,0,RagWing Aircraft Designs,RagWing Aircraft Designs (also called the Rag...


In [ ]:

def get_lemma(word):
  lemma = wn.morphy(word)
  if lemma is None:
    return word
  else:
    return lemma



In [ ]:
def get_lemma2(word):
  return WordNetLemmatizer().lemmatize(word)

In [ ]:

nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
parser = English()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def prepare_text_for_lda(text):
  #print(":)")
  #print(text)
  tokens = tokenize(text)
  tokens = [token for token in tokens if len(token) > 4]
  tokens = [token for token in tokens if token not in en_stop]
  #print(tokens)
  return tokens

In [ ]:
'''
def prepare_text_for_lda(tokens):
  #print(":)")
  #print(text)
  #tokens = tokenize(text)
  tokens = [token for token in tokens if len(token) > 4]
  tokens = [token for token in tokens if token not in en_stop]
  #print(tokens)
  return tokens'''

'\ndef prepare_text_for_lda(tokens):\n  #print(":)")\n  #print(text)\n  #tokens = tokenize(text)\n  tokens = [token for token in tokens if len(token) > 4]\n  tokens = [token for token in tokens if token not in en_stop]\n  #print(tokens)\n  return tokens'

In [ ]:
def tokenize(text):
  lda_tokens = []
  tokens = parser(text)
  for token in tokens:
    if token.orth_.isspace():
      continue
    elif token.like_url:
      lda_tokens.append('URL')
    elif token.orth_.startswith('@'):
      lda_tokens.append('SCREEN_NAME')
    else:
      lda_tokens.append(token.lower_)
    return lda_tokens

In [ ]:
'''
def tokenize_content_column():
  text_data = []
  for index, row in df_train.iterrows():
    print(str(row[2]))
    if random.random() > 0.99:
      tokens = prepare_text_for_lda(str(row[2]))
      text_data.append(tokens)
  return text_data'''



'\ndef tokenize_content_column():\n  text_data = []\n  for index, row in df_train.iterrows():\n    print(str(row[2]))\n    if random.random() > 0.99:\n      tokens = prepare_text_for_lda(str(row[2]))\n      text_data.append(tokens)\n  return text_data'

In [ ]:
def tokenize_content_column():
  #df_train.apply(lambda x: print("{} - {}".format(x['title'], x['content'])), axis = 1)
  token_data = []
  df = df_train
  df['tokenized_sents'] = df_train.apply(lambda row: nltk.word_tokenize(row['content']), axis = 1)
  #print(token_data)
  return df


In [ ]:
#df_tokenized = tokenize_content_column()

In [ ]:
def prepare_tokenized_column():
  #df_train.apply(lambda x: print("{} - {}".format(x['title'], x['content'])), axis = 1)
  token_data = []
  df = df_train
  df['tokenized_sents'] = df.apply(lambda row: prepare_text_for_lda(row['content'][0:5000]), axis = 1)
  #print(token_data)
  return df


In [ ]:
df_tokenized = prepare_tokenized_column()

In [ ]:
text_data = df_tokenized['tokenized_sents'].to_list()

In [ ]:
dictionary = corpora.Dictionary(text_data)

In [ ]:
dictionary

In [ ]:
corpus = [dictionary.doc2bow(text) for text in text_data]

In [ ]:
#FIXME save to a specific folder  within directory
pickle.dump(corpus, open('corpus1.pkl', 'wb'))
dictionary.save('dictionary.gensim') 

In [ ]:
# 9.23.2021 at 8:41PM EST -ad
#FixMe apply EDA to only a smaller subset of training data based upon Cronbach alpha sampling and threshold recommendations
# this logic is taking over 10 minutes to run due the size of the current sample 
# GOOGLE COLAB LIMITS REACHED during this iteration; restart notebook at this checkpoint

NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word = dictionary, passes = 15)
ldamodel.save('model5.gensim')

In [ ]:
topics = ldamodel.print_topics(num_words = 4)
for topic in topics:
  print(topic)

(0, '0.011*"death" + 0.009*"three" + 0.006*"beyond" + 0.006*"conan"')
(1, '0.011*"journal" + 0.005*"international" + 0.004*"green" + 0.004*"double"')
(2, '0.017*"black" + 0.006*"young" + 0.004*"nature" + 0.004*"world"')
(3, '0.008*"blood" + 0.008*"little" + 0.006*"white" + 0.006*"first"')
(4, '0.011*"american" + 0.007*"night" + 0.006*"dragon" + 0.006*"tales"')


In [ ]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus1.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

In [ ]:
lda_display = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary, sort_topics = False)
pyLDAvis.display(lda_display)

In [ ]:
#text_data = prepare_text_for_lda(list(itertools.chain.from_iterable(df_tokenized['tokenized_sents_prep'].to_list())))

# Task 1: Analyze the data sets and share relevant insights which influence your model design decisions
* Please analyze the datasets and share your code with us.
* What do you observe about the two dataframes? 
* Is the test dataset representative of the training dataset? If not, what * would have to change?
* Based on your analysis, which model architecture are you preferring? In a few words, please explain why?

### Tips:
* You can use any 3rd party package which you think is useful


# Task 2: Train a model to classify the content regarding the categories
* Implement an ML model which can classify the given content feature (content only, not title) into one of the 14 categories
* In a few words, please explain your rationality for your model architecture.
* Please address the limitations of your chosen approach? Are there any major upsides/downsides to your approach?

### Tips:
* Enable the GPU support via Runtime >> Change Runtime Type >> Select GPU
* If the training time exceeds your available time, downsample the dataset
* You can test your machine learning model with the provided example string.
* You can use an ML framework of your choice (e.g. Tensorflow, PyTorch, Scikit). Your choice of ML technique won't impact the exercise grading.
* Implement the classification task by implementing an ML approach. The choice of ML approach won't impact the exercise grading. However, expect questions around your chosen ML approach in follow up conversations.
* Applying transfer learning is totally ok. You can use pre-trained models from tf.hub, Pytorch.hub or huggingface. However, refrain from using fully trained (zero-shot learning) models. We are interested in your solution.
* Time box this task! Don't spend no more than 2 hours on this task. In case, your model isn't converging - no worries. In such a case, explain to us why it might have happened. Not all ML experiments at Digits succeed. But it is critical to understand why a chosen path didn't succeed, so that we can avoid similar paths next time.


### Test your ML model

Which category is your model predicting for the test string below?

In [ ]:
import numpy as np

test_text = "Digits is a fin-tech company offering a visual, machine learning-powered expense monitoring dashboard for startups and small businesses."

# Adjust the model call depending on your ML framework
rs = model.predict([test_text])

print(np.argmax(rs))

NameError: ignored

# Task 3: Extract an Embedding
* Update your ML model from task 2 to extract an embedding model for a provided content feature?
* Which dimensionality of your embedding would you choose? Please explain why.

### Tips:
* We don’t anticipate that you train a new model for this task
* Try to extract the embedding model from your previously trained ML model


# Task 4: Visualize your extracted embeddings
* Visualize 1000 content feature samples from your test dataset with the extracted embedding model from Task 3?
* What do you observe?
* Share a screenshot of your visualization

### Tips:
* You can use any visualization of your choice (e.g. matplotlib, plotly, Tensorboard)
* You can use any dimensionality reduction algorithm (e.g. PCA)
